In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
import numpy as np
import os
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

IMG_SIZE = 128
BATCH_SIZE = 32
EPOCHS = 5

def load_images_from_folder(folder_path):
    images = []
    labels = []
    class_names = sorted([d for d in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, d))])

    print(f"Found class names: {class_names}")

    for class_index, class_name in enumerate(class_names):
        class_path = os.path.join(folder_path, class_name)
        print(f"Processing class folder: {class_path}")
        if not os.path.isdir(class_path):
            print(f"Skipping {class_path} because it's not a directory.")
            continue
        for img_name in os.listdir(class_path):
            if img_name.lower().endswith('.jpg'):
                img_path = os.path.join(class_path, img_name)
                try:
                    image = Image.open(img_path)
                    image = image.convert("RGB")
                    image = image.resize((IMG_SIZE, IMG_SIZE))
                    image = np.array(image) / 255.0
                    images.append(image)
                    labels.append(class_index)
                except Exception as e:
                    print(f"Error loading image {img_path}: {e}")

    images = np.array(images)
    labels = np.array(labels)

    print(f"Loaded {len(images)} images and {len(labels)} labels.")
    print(f"Class names: {class_names}")

    return images, labels, class_names

def preprocess_and_split(images, labels):
    if len(images) == 0:
        raise ValueError("No images were loaded. Please check the folder path and ensure it contains valid JPG images.")

    X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
    train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(1000).batch(BATCH_SIZE)
    test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(BATCH_SIZE)

    return train_dataset, test_dataset

def build_model(num_classes):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

def train_and_evaluate(model, train_dataset, test_dataset):
    history = model.fit(train_dataset, epochs=EPOCHS, validation_data=test_dataset)
    return history

def calculate_accuracy(model, test_dataset):
    test_images = []
    test_labels = []
    for images, labels in test_dataset:
        test_images.extend(images.numpy())
        test_labels.extend(labels.numpy())

    test_images = np.array(test_images)
    test_labels = np.array(test_labels)

    predictions = model.predict(test_images)
    predicted_labels = np.argmax(predictions, axis=1)

    accuracy = accuracy_score(test_labels, predicted_labels)
    print(f"Test Accuracy: {accuracy:.2f}")
    return accuracy

def predict_disease(image_path, model, class_names):
    image = Image.open(image_path)
    image = image.convert("RGB")
    image = image.resize((IMG_SIZE, IMG_SIZE))
    image = np.array(image) / 255.0
    image = np.expand_dims(image, axis=0)

    predictions = model.predict(image)
    predicted_label = np.argmax(predictions)

    disease = class_names[predicted_label]

    solutions = {
        'healthy': 'No disease detected, the plant is healthy.',
        'early_blight': 'Solution: Remove affected leaves, use a fungicide.',
        'late_blight': 'Solution: Destroy infected plants, use a fungicide.',
        'leaf_mold': 'Solution: Improve air circulation, use fungicide.',
        'powdery_mildew': 'Solution: Apply sulfur-based fungicide.',
        'septoria_leaf_spot': 'Solution: Remove infected leaves, use fungicide.'
    }

    solution = solutions.get(disease, "Solution not available")

    return disease, solution

def get_image_and_predict(model, class_names):
    image_path = input("Please enter the path to the plant image: ")
    if os.path.exists(image_path):
        disease, solution = predict_disease(image_path, model, class_names)
        print(f"Disease: {disease}")
        print(f"Solution: {solution}")
    else:
        print("Invalid file path. Please try again.")

folder_path = r"/content/drive/MyDrive/PlantVillage"

if os.path.exists(folder_path):
    images, labels, class_names = load_images_from_folder(folder_path)
    train_dataset, test_dataset = preprocess_and_split(images, labels)
    model = build_model(num_classes=len(class_names))
    train_and_evaluate(model, train_dataset, test_dataset)
    calculate_accuracy(model, test_dataset)
    model.save('plant_disease_model.keras')
    get_image_and_predict(model, class_names)
else:
    print("Invalid folder path. Please try again.")


Found class names: ['Pepper__bell___Bacterial_spot']
Processing class folder: /content/drive/MyDrive/PlantVillage/Pepper__bell___Bacterial_spot
Loaded 142 images and 142 labels.
Class names: ['Pepper__bell___Bacterial_spot']
Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


4/4 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 2/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 3/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 4/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 5/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00


/usr/local/lib/python3.10/dist-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (29, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 348ms/step
Test Accuracy: 1.00
Please enter the path to the plant image: /content/Potato-leaf-blight.webp


/usr/local/lib/python3.10/dist-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 787ms/step
Disease: Pepper__bell___Bacterial_spot
Solution: Solution not available
